<a href="https://colab.research.google.com/github/AdityaMayukhSom/hallucination-detection-pipeline/blob/main/DatasetGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

### Generate Smaller Dataset For Easier Processing

Loaded the original dataset from huggingface to generate a sliced smalled version which is easier to process.

In [ ]:
%%capture
!pip install datasets

In [ ]:
import os
from pathlib import Path
from datasets import load_dataset, load_from_disk, Dataset

In [ ]:
dataset_name = "TRnlp/MixSub"
ori_dataset_dir = "/content/drive/MyDrive/Datasets/MixSub/Original"
dup_dataset_dir = "/content/drive/MyDrive/Datasets/MixSub/Duplicate"
dup_csv_file_name = "duplicate_mixsub.csv"
num_rows_in_dup = 20

If we do not mention `split` argument while in `load_dataset` function, three seperate folders will be downloaded for `train`, `test` and `validation`. In that case, the `dataset` object generated can be indexed for the three splits which will then return the dataset which contains the actual rows. Mentioning which split to load during in `load_dataset` will return the actual dataset on which we can index with the attribute names.

In [ ]:
if not os.path.exists(ori_dataset_dir):
    dataset = load_dataset(dataset_name)
    dataset.save_to_disk(ori_dataset_dir)
else:
    dataset = load_from_disk(ori_dataset_dir)

In [ ]:
train_ds = dataset['train']
test_ds = dataset['test']
validation_ds = dataset['validation']

In [ ]:
# Selects num rows from the original and generates a small dataset
# with first `num_rows_in_dup` entries.
small_ds = train_ds.select(range(num_rows_in_dup))

# Converts the huggingface Datasets type to Pandas Dataframe type
small_pd_df = small_ds.to_pandas()

small_pd_df.head()

Checks whether the path to save the small dataset exists or not, if not, creates the directory to in which it will save the generated file. It writes the small dataset in CSV format, which is different from the original Apache Arrow format.

In [ ]:
if not os.path.exists(dup_dataset_dir):
    Path(dup_dataset_dir).mkdir(parents=True, exist_ok=True)

dup_ds_path = os.path.join(dup_dataset_dir, dup_csv_file_name)
small_pd_df.to_csv(dup_ds_path, index=True)

### Genrate Hallucinated Dataset

#### Unsloth Installation Command Generator

In [ ]:
# We have to check which Torch version for Xformers (2.3 -> 0.0.27)
import torch
from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(torch.__version__) < V("2.4.0") else "xformers"

In [ ]:
try:
    import torch
except:
    raise ImportError('Install torch via `pip install torch`')
from packaging.version import Version as V
v = V(torch.__version__)
cuda = str(torch.version.cuda)
is_ampere = torch.cuda.get_device_capability()[0] >= 8
if cuda != "12.1" and cuda != "11.8" and cuda != "12.4": raise RuntimeError(f"CUDA = {cuda} not supported!")
if   v <= V('2.1.0'): raise RuntimeError(f"Torch = {v} too old!")
elif v <= V('2.1.1'): x = 'cu{}{}-torch211'
elif v <= V('2.1.2'): x = 'cu{}{}-torch212'
elif v  < V('2.3.0'): x = 'cu{}{}-torch220'
elif v  < V('2.4.0'): x = 'cu{}{}-torch230'
elif v  < V('2.5.0'): x = 'cu{}{}-torch240'
elif v  < V('2.6.0'): x = 'cu{}{}-torch250'
else: raise RuntimeError(f"Torch = {v} too new!")
x = x.format(cuda.replace(".", ""), "-ampere" if is_ampere else "")
print(f'pip install --upgrade pip && pip install "unsloth[{x}] @ git+https://github.com/unslothai/unsloth.git"')

In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install --upgrade pip
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton
!pip install "unsloth[cu124-torch250] @ git+https://github.com/unslothai/unsloth.git"

#### Create Unsloth Model

In [ ]:
import os
import torch
import pandas as pd
from pathlib import Path
from unsloth import FastLanguageModel

In [ ]:
model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"
max_seq_length = 2048
load_in_4bit = True
dtype = None
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    load_in_4bit = load_in_4bit,
    dtype = dtype
)

#### Setup Model For Inference

In [ ]:
FastLanguageModel.for_inference(model)

#### Hallucinated Highlight Generator

In [ ]:
instruction = """
You're instructed to generate scientifically inaccurate or hallucinated highlights of the provided passage
only without additional sentences like headings, introductions, or text before or after the generated output
as the output will be directly used as highlight in a custom dataset. The highlight should sound plausible
but contain incorrect information.Generate 3-5 concise highlight points from the provided research paper abstract,
covering key contributions, methods, and outcomes. Each point should contain 10 to 15 words only. Return the
points in plain text format without bullets.

No Additional Commentary: Exclude lines like "Here are 3-5 concise highlight points".
"""

def generate_hallucinated_highlights(abstract: str) -> str:
    row_json = [
        {"role": "system", "content": instruction},
        {"role": "user", "content": abstract}
    ]

    model_prompt = tokenizer.apply_chat_template(row_json, tokenize=False, add_generation_prompt=True)
    model_inputs = tokenizer(model_prompt, return_tensors='pt', padding=True, truncation=True).to(device)
    model_outputs = model.generate(**model_inputs, max_new_tokens=150, num_return_sequences=1)
    decoded_text = tokenizer.batch_decode(model_outputs, skip_special_tokens=False)
    hallucination = decoded_text[0].split("<|start_header_id|>assistant<|end_header_id|>")[1].strip().split("<|eot_id|>")[0].strip()
    return hallucination

#### Dataset Loader For Hallucinated Dataset

In [ ]:
ds_path = Path(os.path.join(dup_dataset_dir, dup_csv_file_name))
custom_df = pd.read_csv(ds_path)
abstract_ds = Dataset.from_pandas(custom_df['Abstract'].to_frame())
hallucination_dataset = abstract_ds.map(generate_hallucinated_highlights, batched=True,)

# Step 5: Save updated dataset
# updated_dataset.save_to_disk("./mixsub_with_hallucinations")
print("Dataset updated and saved successfully!")